In [1]:
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import time
import re
import string
import json
import pickle
from collections import defaultdict

%matplotlib inline

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

from bs4 import BeautifulSoup
# The "requests" library makes working with HTTP requests easier
# than the built-in urllib libraries.
import requests
from urllib2 import urlopen

#Preliminary Data Cleaning

In [2]:
data=pd.read_csv("../data/anime_info.csv",sep='\t', index_col=0)

In [3]:
data.shape

(7281, 11)

In [4]:
data[1:100]

,altname,atype,categories,date,date_pub,description,name,rank,rating,related,studio
+A-Channel,NaN,\nDVD Special (11 eps x 2 min),"[Comedy, Slice of Life, Based on a Manga]",NaN,2011,There are many untold stories in the lives of ...,+A-Channel,"\n Rank #2,398","3.334 of 5 from 1,192 votes","[A-Channel, A-Channel + smile]",Studio Gokumi
.hack//G.U. Returner,NaN,\nOVA (1 ep),"[MMORPG, RPG, Virtual Reality]",NaN,2007,"""There is something I wish to entrust to you; ...",.hack//G.U. Returner,"\n Rank #2,504","3.291 of 5 from 2,021 votes","[.hack//SIGN, .hack//LIMINALITY, .hack//Legend...",NaN
.hack//G.U. Trilogy,NaN,\nMovie (1 ep x 93 min),"[CG Animation, MMORPG, RPG, Virtual Reality]",NaN,2008,The World is an ordinary online game – or at l...,.hack//G.U. Trilogy,"\n Rank #1,576","3.671 of 5 from 2,814 votes","[.hack//SIGN, .hack//LIMINALITY, .hack//Legend...",NaN
.hack//G.U. Trilogy Parody,NaN,\nDVD Special (1 ep x 6 min),"[Comedy, MMORPG, RPG, Virtual Reality]",NaN,2008,NaN,.hack//G.U. Trilogy Parody,"\n Rank #3,993",2.691 of 5 from 714 votes,"[.hack//SIGN, .hack//LIMINALITY, .hack//Legend...",NaN
.hack//GIFT,NaN,\nOVA (1 ep),"[Chibi, MMORPG, RPG, Virtual Reality]",NaN,2003,"As thanks for playing in The World, its makers...",.hack//GIFT,"\n Rank #4,108","2.646 of 5 from 2,721 votes","[.hack//SIGN, .hack//LIMINALITY, .hack//Legend...",BEE TRAIN
.hack//LIMINALITY,NaN,\nOVA (4 eps),NaN,NaN,2002,"While characters within the MMORPG entitled ""T...",.hack//LIMINALITY,"\n Rank #3,073","3.064 of 5 from 4,253 votes","[.hack//SIGN, .hack//Legend Of The Twilight, ....",BEE TRAIN
.hack//Legend Of The Twilight,NaN,\nTV (12 eps),"[Sci Fi, MMORPG, RPG, Virtual Reality]",NaN,2003,Shuugo and Rena think they're just two sibling...,.hack//Legend Of The Twilight,"\n Rank #3,039","3.078 of 5 from 7,252 votes","[.hack//SIGN, .hack//LIMINALITY, .hack//SIGN -...",BEE TRAIN
.hack//Quantum,NaN,\nOVA (3 eps),"[Action, Mystery, Sci Fi, MMORPG]",NaN,2010,Asumi Aida is a cheerful high school student w...,.hack//Quantum,\n Rank #974,"3.933 of 5 from 2,328 votes","[.hack//SIGN, .hack//LIMINALITY, .hack//Legend...",Kinema Citrus
.hack//Quantum Specials,NaN,\nDVD Special (3 eps x 3 min),NaN,NaN,2011,NaN,.hack//Quantum Specials,"\n Rank #3,441",2.916 of 5 from 311 votes,"[.hack//SIGN, .hack//LIMINALITY, .hack//Legend...",Kinema Citrus
.hack//Roots,NaN,\nTV (26 eps x 24 min),"[MMORPG, RPG, Virtual Reality]",Spring 2006,2006,Years after the fantasy MMORPG known as 'The W...,.hack//Roots,"\n Rank #2,198","3.414 of 5 from 6,266 votes","[.hack//SIGN, .hack//LIMINALITY, .hack//Legend...",BEE TRAIN


We needed to do some more cleaning up as some of the data was not in the desired type. Specifically, the `categories` column was in a string and not in a list, so we had to convert that. This was stored in `categores_list`. 

In [5]:
def categories_list(s):
    if type(s) == float:
        return []
    else:
        return s.translate(None, "[]").split(', ')

In [6]:
l = []
for a in data.categories:
    l.append(categories_list(a))
data['categories_list'] = l

In [7]:
data.head(5)

,altname,atype,categories,date,date_pub,description,name,rank,rating,related,studio,categories_list
(Making of) Evangelion: Another Impact,NaN,\n Web (1 ep x 6 min),"[Sci Fi, CG Animation, Japan Animator's Exhibi...",NaN,2015,"This episode is a making film of #12 ""evangeli...",(Making of) Evangelion: Another Impact,"\n Rank #5,996",1.743 of 5 from 153 votes,"[Neon Genesis Evangelion, Neon Genesis Evangel...",Studio Khara,"[Sci Fi, CG Animation, Japan Animator's Exhibi..."
+A-Channel,NaN,\nDVD Special (11 eps x 2 min),"[Comedy, Slice of Life, Based on a Manga]",NaN,2011,There are many untold stories in the lives of ...,+A-Channel,"\n Rank #2,398","3.334 of 5 from 1,192 votes","[A-Channel, A-Channel + smile]",Studio Gokumi,"[Comedy, Slice of Life, Based on a Manga]"
.hack//G.U. Returner,NaN,\nOVA (1 ep),"[MMORPG, RPG, Virtual Reality]",NaN,2007,"""There is something I wish to entrust to you; ...",.hack//G.U. Returner,"\n Rank #2,504","3.291 of 5 from 2,021 votes","[.hack//SIGN, .hack//LIMINALITY, .hack//Legend...",NaN,"[MMORPG, RPG, Virtual Reality]"
.hack//G.U. Trilogy,NaN,\nMovie (1 ep x 93 min),"[CG Animation, MMORPG, RPG, Virtual Reality]",NaN,2008,The World is an ordinary online game – or at l...,.hack//G.U. Trilogy,"\n Rank #1,576","3.671 of 5 from 2,814 votes","[.hack//SIGN, .hack//LIMINALITY, .hack//Legend...",NaN,"[CG Animation, MMORPG, RPG, Virtual Reality]"
.hack//G.U. Trilogy Parody,NaN,\nDVD Special (1 ep x 6 min),"[Comedy, MMORPG, RPG, Virtual Reality]",NaN,2008,NaN,.hack//G.U. Trilogy Parody,"\n Rank #3,993",2.691 of 5 from 714 votes,"[.hack//SIGN, .hack//LIMINALITY, .hack//Legend...",NaN,"[Comedy, MMORPG, RPG, Virtual Reality]"


We now split out the categories as their own individual column so that this can be quanitified and measured in future analaysis of this dataset. It turns out that we have a total of 233 different categories 

In [8]:
categories = set()
for anime in data.categories_list:
        categories.update(i for i in anime)
categories = list(categories)
categories.sort()

for c in categories:
    data[c] = [c in anime for anime in data.categories_list]
    

In [9]:
data.head(50)

,altname,atype,categories,date,date_pub,description,name,rank,rating,related,studio,categories_list,Abstract,Action,Adventure,Age Gap,Age Transformation,Agriculture,Alice in Wonderland,Aliens,America,Androids,Angels,Animal Abuse,Animal Protagonists,Anime Bancho,Anime Industry,Anime Mirai,Arranged Marriage,Assassins,Astronauts,Aviation,Baseball,Based on a Card Game,Based on a Comic Book,Based on a Doujinshi,Based on a Fairy Tale,Based on a Light Novel,Based on a Manga,Based on a Movie,Based on a Non-Fiction Work,Based on a Novel,Based on a TV Series,Based on a Video Game,Based on a Visual Novel,Based on an Anime,Based on an Eroge,Based on an Otome Game,Basketball,Black and White,...,Science,Seinen,Sexual Abuse,Sexual Content,Shinigami,Shorts,Shoujo,Shoujo-ai,Shounen,Shounen-ai,Slice of Life,Soccer,Social Media,Space Opera,Space Pirates,Sports,Steampunk,Stop Motion Animation,Student-Teacher Relationship,Sudden Girlfriend Appearance,Suicide,Supernatural,Superpowers,Survival,Swordplay,Tennis,Terrorism,Thieves,Thriller,Time Travel,Tournaments,Trains,Treasure Hunters,Tsundere,Urban Legend,Vampires,Violence,Virtual Reality,Vocaloid,War,Western,Work Life,World Masterpiece Theater,World War 2,Yaoi,Yaoi Hand Syndrome,Youkai,Yuri,Zombies,noitaminA
(Making of) Evangelion: Another Impact,NaN,\n Web (1 ep x 6 min),"[Sci Fi, CG Animation, Japan Animator's Exhibi...",NaN,2015,"This episode is a making film of #12 ""evangeli...",(Making of) Evangelion: Another Impact,"\n Rank #5,996",1.743 of 5 from 153 votes,"[Neon Genesis Evangelion, Neon Genesis Evangel...",Studio Khara,"[Sci Fi, CG Animation, Japan Animator's Exhibi...",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
+A-Channel,NaN,\nDVD Special (11 eps x 2 min),"[Comedy, Slice of Life, Based on a Manga]",NaN,2011,There are many untold stories in the lives of ...,+A-Channel,"\n Rank #2,398","3.334 of 5 from 1,192 votes","[A-Channel, A-Channel + smile]",Studio Gokumi,"[Comedy, Slice of Life, Based on a Manga]",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
.hack//G.U. Returner,NaN,\nOVA (1 ep),"[MMORPG, RPG, Virtual Reality]",NaN,2007,"""There is something I wish to entrust to you; ...",.hack//G.U. Returner,"\n Rank #2,504","3.291 of 5 from 2,021 votes","[.hack//SIGN, .hack//LIMINALITY, .hack//Legend...",NaN,"[MMORPG, RPG, Virtual Reality]",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
.hack//G.U. Trilogy,NaN,\nMovie (1 ep x 93 min),"[CG Animation, MMORPG, RPG, Virtual Reality]",NaN,2008,The World is an ordinary online game – or at l...,.hack//G.U. Trilogy,"\n Rank #1

In [10]:
len(categories)

233

In [11]:
categories

['Abstract',
 'Action',
 'Adventure',
 'Age Gap',
 'Age Transformation',
 'Agriculture',
 'Alice in Wonderland',
 'Aliens',
 'America',
 'Androids',
 'Angels',
 'Animal Abuse',
 'Animal Protagonists',
 'Anime Bancho',
 'Anime Industry',
 'Anime Mirai',
 'Arranged Marriage',
 'Assassins',
 'Astronauts',
 'Aviation',
 'Baseball',
 'Based on a Card Game',
 'Based on a Comic Book',
 'Based on a Doujinshi',
 'Based on a Fairy Tale',
 'Based on a Light Novel',
 'Based on a Manga',
 'Based on a Movie',
 'Based on a Non-Fiction Work',
 'Based on a Novel',
 'Based on a TV Series',
 'Based on a Video Game',
 'Based on a Visual Novel',
 'Based on an Anime',
 'Based on an Eroge',
 'Based on an Otome Game',
 'Basketball',
 'Black and White',
 'Blackmail',
 'Board Games',
 'Boarding House',
 'Body Sharing',
 'Body Swapping',
 'Bounty Hunters',
 'Boxing',
 'Bullying',
 'CG Animation',
 'Cafe',
 'Card Battles',
 'Cars',
 'Chibi',
 'Childcare',
 'Classical Music',
 'Collections',
 'Combining Mecha',
 '

In [12]:
for c in categories:
    g = data.groupby(c)
    g
    break

In [13]:
data.groupby('Dragons')


#Reviews Analysis

In [2]:
reviewsdf=pd.read_csv("all_reviews.csv",sep='\t', index_col=0)

In [3]:
reviewsdf.head(5)

,animation,anime,anime_name,character,name,overall,reviews,sound,story
0,\n10/10,to-aru-majutsu-no-index-10th-anniversary-pv,To Aru Majutsu no Index 10th Anniversary PV,\n10/10,SogiitaGunhaGUTS,\n10/10,Well it's a short animation to celebrate Kamac...,\n?/10,\n?/10
1,\n5/10,dokkoida,Dokkoida?!,\n6/10,angelsreviews,\n5/10,NaN,\n5/10,\n8/10
2,\n9/10,inazuma-eleven,Inazuma Eleven,\n9/10,xxjacobxx3,\n9/10,first of all: I agree with ﻿﻿﻿﻿luminex's revie...,\n9/10,\n9/10
3,\n8/10,inazuma-eleven,Inazuma Eleven,\n6/10,Lumineux,\n7/10,Story: 6,\n8/10,\n6/10
4,\n8/10,brigadoon-marin-to-melan,Brigadoon Marin to Melan,\n8/10,roriconfan,\n8/10,NaN,\n9/10,\n8/10


Let us do some textual analysis of some of the reviews. First, we make sure Spark is working. 

In [4]:
import findspark
findspark.init()
print findspark.find()

/home/vagrant/spark


In [5]:
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)

In [6]:
import sys
rdd = sc.parallelize(xrange(10),10)
rdd.map(lambda x: sys.version).collect()

['2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 

In [7]:
sys.version

'2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'

In [8]:
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)

Some initial information such as the number of reviews and columns.

In [9]:
print "How many reviews do we have?", reviewsdf.count()

How many reviews do we have? animation     17194
anime         17194
anime_name    17194
character     17194
name          17194
overall       17194
reviews        9382
sound         17194
story         17194
dtype: int64


For the sake of this section, we only care about those reviews that have textual reviews along with the rating itself. 

In [10]:
reviewsdf['reviews'].replace('', np.nan, inplace=True)
reviewsdf.dropna(subset=['reviews'], inplace=True)
reviewsdf.count()

animation     9382
anime         9382
anime_name    9382
character     9382
name          9382
overall       9382
reviews       9382
sound         9382
story         9382
dtype: int64

In [11]:
reviewsdf

,animation,anime,anime_name,character,name,overall,reviews,sound,story
0,\n10/10,to-aru-majutsu-no-index-10th-anniversary-pv,To Aru Majutsu no Index 10th Anniversary PV,\n10/10,SogiitaGunhaGUTS,\n10/10,Well it's a short animation to celebrate Kamac...,\n?/10,\n?/10
2,\n9/10,inazuma-eleven,Inazuma Eleven,\n9/10,xxjacobxx3,\n9/10,first of all: I agree with ﻿﻿﻿﻿luminex's revie...,\n9/10,\n9/10
3,\n8/10,inazuma-eleven,Inazuma Eleven,\n6/10,Lumineux,\n7/10,Story: 6,\n8/10,\n6/10
6,\n9/10,brigadoon-marin-to-melan,Brigadoon Marin to Melan,\n9/10,xxjacobxx3,\n8.5/10,Warning: This review has a few things that cou...,\n9/10,\n8/10
8,\n7/10,brigadoon-marin-to-melan,Brigadoon Marin to Melan,\n9/10,ChillWinterheart,\n10/10,Most anime series can be neatly categorized in...,\n9/10,\n8/10
11,\n?/10,shakugan-no-shana-ova,Shakugan no Shana OVA,\n?/10,WolfAngelus,\n8/10,This one is actually good compared to the spec...,\n?/10,\n?/10
14,\n7/10,hack-quantum,.hack//Quantum,\n7/10,Bobnotk,\n8/10,As a fan of the entire .Hack series I beleive ...,\n8/10,\n8/10
17,\n7/10,wizardry,Wizardry,\n8/10,Pauwolf,\n7/10,"Inspired by an old videogame, this 50 minutes ...",\n7/10,\n6/10
23,\n7/10,haibane-renmei,Haibane Renmei,\n8/10,haize78,\n9/10,Haibane Renmei is a beautiful anime and I real...,\n7/10,\n9/10
25,\n8/10,haibane-renmei,Haibane Renmei,\n8.5/10,LimeCultivist,\n9.2/10,The story of Haibane Renmei was created by Yos...,\n8.5/10,\n9.5/10


We now also clean the `overall` column to make it more quantifiable.

In [12]:
reviewsdf['overall'] = reviewsdf['overall'].str.strip('\n/').map(lambda x: float(x[:-4]))
reviewsdf.head()

,animation,anime,anime_name,character,name,overall,reviews,sound,story
0,\n10/10,to-aru-majutsu-no-index-10th-anniversary-pv,To Aru Majutsu no Index 10th Anniversary PV,\n10/10,SogiitaGunhaGUTS,10.0,Well it's a short animation to celebrate Kamac...,\n?/10,\n?/10
2,\n9/10,inazuma-eleven,Inazuma Eleven,\n9/10,xxjacobxx3,9.0,first of all: I agree with ﻿﻿﻿﻿luminex's revie...,\n9/10,\n9/10
3,\n8/10,inazuma-eleven,Inazuma Eleven,\n6/10,Lumineux,7.0,Story: 6,\n8/10,\n6/10
6,\n9/10,brigadoon-marin-to-melan,Brigadoon Marin to Melan,\n9/10,xxjacobxx3,8.5,Warning: This review has a few things that cou...,\n9/10,\n8/10
8,\n7/10,brigadoon-marin-to-melan,Brigadoon Marin to Melan,\n9/10,ChillWinterheart,10.0,Most anime series can be neatly categorized in...,\n9/10,\n8/10


To better understand how users are reponsding to specific anime, we want to figure out whether each of the reviews are positive or negative with respect to the user's average rating. We then create a `response` column as well as make a `resarray` so that we can use it for further analysis. The item will be True is the rating is greater than or equal to the user's average rating and False otherwise. 

In [13]:
useravgdf = reviewsdf.groupby('name').overall.mean()

In [14]:
def user_average(row):
    return useravgdf[row['name']]

In [15]:
reviewsdf['user_avg'] = reviewsdf.apply(lambda row: user_average(row), axis=1)
reviewsdf['responses'] = reviewsdf.apply((lambda row: row.overall >= row.user_avg), axis=1)

In [16]:
reviewsdf.head()

,animation,anime,anime_name,character,name,overall,reviews,sound,story,user_avg,responses
0,\n10/10,to-aru-majutsu-no-index-10th-anniversary-pv,To Aru Majutsu no Index 10th Anniversary PV,\n10/10,SogiitaGunhaGUTS,10.0,Well it's a short animation to celebrate Kamac...,\n?/10,\n?/10,10.000000,True
2,\n9/10,inazuma-eleven,Inazuma Eleven,\n9/10,xxjacobxx3,9.0,first of all: I agree with ﻿﻿﻿﻿luminex's revie...,\n9/10,\n9/10,7.509302,True
3,\n8/10,inazuma-eleven,Inazuma Eleven,\n6/10,Lumineux,7.0,Story: 6,\n8/10,\n6/10,8.045455,False
6,\n9/10,brigadoon-marin-to-melan,Brigadoon Marin to Melan,\n9/10,xxjacobxx3,8.5,Warning: This review has a few things that cou...,\n9/10,\n8/10,7.509302,True
8,\n7/10,brigadoon-marin-to-melan,Brigadoon Marin to Melan,\n9/10,ChillWinterheart,10.0,Most anime series can be neatly categorized in...,\n9/10,\n8/10,10.000000,True


In [21]:
#reviewsdf.to_json("../data/final_all_reviews.json")

In [17]:
revdf = sqlsc.createDataFrame(reviewsdf)
#revdf.cache()

In [18]:
revdf.cache()

DataFrame[animation: string, anime: string, anime_name: string, character: string, name: string, overall: double, reviews: string, sound: string, story: string, user_avg: double, responses: boolean]

In [19]:
resparray = revdf.map(lambda x: x.overall >= x.user_avg).collect()

Let us now calculate the prior probabiilties for positive and negative.

In [20]:
priorp = np.mean(resparray)
priorn = 1 - priorp
priorp, priorn

(0.67469622681730979, 0.32530377318269021)

In [21]:
revdf.show(5)

+---------+--------------------+--------------------+---------+----------------+-------+--------------------+------+------+-----------------+---------+
|animation|               anime|          anime_name|character|            name|overall|             reviews| sound| story|         user_avg|responses|
+---------+--------------------+--------------------+---------+----------------+-------+--------------------+------+------+-----------------+---------+
|  
10/10 |to-aru-majutsu-no...|To Aru Majutsu no...|  
10/10 |SogiitaGunhaGUTS|   10.0|Well it's a short...|
?/10 |
?/10 |             10.0|     true|
|   
9/10 |      inazuma-eleven|      Inazuma Eleven|   
9/10 |      xxjacobxx3|    9.0|first of all: I a...|
9/10 |
9/10 |7.509302325581395|     true|
|   
8/10 |      inazuma-eleven|      Inazuma Eleven|   
6/10 |        Lumineux|    7.0|            Story: 6|
8/10 |
6/10 |8.045454545454545|    false|
|   
9/10 |brigadoon-marin-t...|Brigadoon Marin t...|   
9/10 |      xxjacobxx3|    8.5|

#Natural Langauge Processing: Nouns

In [22]:
from pattern.en import parse
from pattern.en import pprint
from pattern.vector import stem, PORTER, LEMMA
punctuation = list('.,;:!?()[]{}`''\"@#$^&*+-|=~_')

from sklearn.feature_extraction import text 
stopwords=text.ENGLISH_STOP_WORDS

import re
regex1=re.compile(r"\.{2,}")
regex2=re.compile(r"\-{2,}")

As in hm5, we implement a `get_parts` function such that it will help simplify sentence parsing. We then find the main words in each sentence so that this will help us when looking at sentiment analysis. 

In [23]:
def get_parts(thetext):
    thetext=re.sub(regex1, ' ', thetext)
    thetext=re.sub(regex2, ' ', thetext)
    nouns=[]
    descriptives=[]
    for i,sentence in enumerate(parse(thetext, tokenize=True, lemmata=True).split()):
        nouns.append([])
        descriptives.append([])
        for token in sentence:
            #print token
            if len(token[4]) >0:
                if token[1] in ['JJ', 'JJR', 'JJS']:
                    if token[4] in stopwords or token[4][0] in punctuation or token[4][-1] in punctuation or len(token[4])==1:
                        continue
                    descriptives[i].append(token[4])
                elif token[1] in ['NN', 'NNS']:
                    if token[4] in stopwords or token[4][0] in punctuation or token[4][-1] in punctuation or len(token[4])==1:
                        continue
                    nouns[i].append(token[4])
    out=zip(nouns, descriptives)
    nouns2=[]
    descriptives2=[]
    for n,d in out:
        if len(n)!=0 and len(d)!=0:
            nouns2.append(n)
            descriptives2.append(d)
    return nouns2, descriptives2

In [30]:
review_parts = revdf.map(lambda x: get_parts(x.reviews))

In [31]:
review_parts.take(1)

[([[u'animation', u'work', u'novelist', u'character', u'series']],
  [[u'short', u'different']])]

In [32]:
%%time
parseout=review_parts.collect()

CPU times: user 192 ms, sys: 7.72 ms, total: 199 ms
Wall time: 1min 16s


In [33]:
import pickle

#with open('tempdata/parseout.pkl', 'wb') as f:
#    pickle.dump(parseout, f)


with open('tempdata/parseout.pkl', 'rb') as f:
    parseout = pickle.load(f)

Create a vocabulary of the words that we have so far.

In [34]:
ldadatardd=sc.parallelize([ele[0] for ele in parseout]).flatMap(lambda l: l)
ldadatardd.cache()
ldadatardd.take(5)

[[u'animation', u'work', u'novelist', u'character', u'series'],
 [u'review', u'deal', u'truth', u'info'],
 [u'series',
  u'kind',
  u'group',
  u'girl',
  u'mecha',
  u'sport',
  u'slice',
  u'life'],
 [u'girl',
  u'series',
  u'example',
  u'girl',
  u'wand',
  u'transformation',
  u'sequence',
  u'brain',
  u'size',
  u'dumpling'],
 [u'kiddy', u'cliche']]

In [35]:
vocabtups = (ldadatardd.flatMap(lambda word: word)
             .map(lambda word: (word, 1))
             .reduceByKey(lambda a, b: a + b)
             .map(lambda (x,y): x)
             .zipWithIndex()
).cache()

In [36]:
vocab=vocabtups.collectAsMap()
id2word=vocabtups.map(lambda (x,y): (y,x)).collectAsMap()
len(vocab.keys())

9030

Create a Corpus in a bag of words form. That is, we now create the lazy RDD documents so that we have a reference to which kins of words are there. 

In [37]:
import collections

def sentence_tuple(sentence):
    d = collections.defaultdict(int)
    for i in sentence:
        d[vocab[i]] += 1
    return d.items()

documents = ldadatardd.map(sentence_tuple)

Take first 5 to make sure we are doing it correctly. It seems like we are! :D So, we'll collect everything and store it in `corpus`.

In [38]:
documents.take(5)

[[(4937, 1), (7578, 1), (172, 1), (6658, 1), (2366, 1)],
 [(1865, 1), (3853, 1), (870, 1), (4959, 1)],
 [(361, 1),
  (172, 1),
  (4531, 1),
  (3151, 1),
  (5267, 1),
  (5236, 1),
  (1686, 1),
  (8157, 1)],
 [(4872, 1),
  (172, 1),
  (5267, 2),
  (8919, 1),
  (89, 1),
  (3386, 1),
  (2963, 1),
  (93, 1),
  (185, 1)],
 [(2941, 1), (3902, 1)]]

In [39]:
corpus=documents.collect()

Now, we carry out the LDA algorithm on the nouns to see the kinds of themes the reviews seem to be focusing on. We will extract 2 topics to see if find any distinct ones. Note that because there warnings when we set `passes = 1`, also tried setting to `passes = 10` to see if the found topics would be potentially different 

In [46]:
import gensim
lda2 = gensim.models.ldamodel.LdaModel(corpus = corpus, id2word = id2word, num_topics = 2, update_every = 1, chunksize = 20000, passes = 1)

In [49]:
lda2.print_topics()

[(0,
  u'0.059*anime + 0.038*character + 0.031*series + 0.030*story + 0.018*time + 0.015*season + 0.014*animation + 0.012*episode + 0.012*plot + 0.011*way'),
 (1,
  u'0.052*anime + 0.034*story + 0.033*character + 0.030*episode + 0.014*thing + 0.013*review + 0.010*movie + 0.010*girl + 0.010*time + 0.008*lot')]

In [48]:
lda2_pass10 = gensim.models.ldamodel.LdaModel(corpus = corpus, id2word = id2word, num_topics = 2, update_every = 1, chunksize = 20000, passes = 10)

In [50]:
lda2_pass10.print_topics()

[(0,
  u'0.096*anime + 0.049*story + 0.040*series + 0.016*review + 0.013*person + 0.010*thing + 0.009*plot + 0.009*episode + 0.007*movie + 0.007*manga'),
 (1,
  u'0.064*character + 0.030*episode + 0.023*time + 0.020*anime + 0.020*animation + 0.019*season + 0.016*story + 0.014*girl + 0.012*way + 0.011*thing')]

In [51]:
for bow in corpus[0:200:10]:
    print bow
    print lda2.get_document_topics(bow)
    print " ".join([id2word[e[0]] for e in bow])
    print "=========================================="

[(4937, 1), (7578, 1), (172, 1), (6658, 1), (2366, 1)]
[(0, 0.73489258360242982), (1, 0.26510741639757018)]
character animation series novelist work
[(2022, 1), (3912, 1), (5674, 1), (2411, 1), (589, 1), (4173, 1), (4308, 1), (5276, 1)]
[(0, 0.31983369797791694), (1, 0.68016630202208306)]
dungeon wizard videogame order minute talk party monster
[(1241, 1), (2761, 1), (1058, 1), (870, 1), (735, 1)]
[(0, 0.10547278611202708), (1, 0.89452721388797285)]
bias anime favorite review largeâ element
[(2761, 1), (1181, 1), (7542, 1)]
[(0, 0.78892226929431164), (1, 0.21107773070568836)]
anime favor season
[(693, 1), (870, 1)]
[(0, 0.59714644859905075), (1, 0.40285355140094925)]
manga review
[(1649, 1), (2945, 1)]
[(0, 0.79540987521841411), (1, 0.20459012478158589)]
line stars.â 
[(8784, 1), (4520, 1), (324, 1), (541, 1), (262, 1)]
[(0, 0.10594861204122803), (1, 0.89405138795877193)]
opening credit nature roll episode
[(8867, 1), (6780, 1)]
[(0, 0.74699576949335167), (1, 0.25300423050664839)]
stud

In [52]:
for bow in corpus[0:200:10]:
    print bow
    print lda2_pass10.get_document_topics(bow)
    print " ".join([id2word[e[0]] for e in bow])
    print "=========================================="

[(4937, 1), (7578, 1), (172, 1), (6658, 1), (2366, 1)]
[(0, 0.23674151773036281), (1, 0.76325848226963711)]
character animation series novelist work
[(2022, 1), (3912, 1), (5674, 1), (2411, 1), (589, 1), (4173, 1), (4308, 1), (5276, 1)]
[(0, 0.23177166811868721), (1, 0.76822833188131279)]
dungeon wizard videogame order minute talk party monster
[(1241, 1), (2761, 1), (1058, 1), (870, 1), (735, 1)]
[(0, 0.91247219981861516), (1, 0.08752780018138491)]
bias anime favorite review largeâ element
[(2761, 1), (1181, 1), (7542, 1)]
[(0, 0.26710103688251452), (1, 0.73289896311748548)]
anime favor season
[(693, 1), (870, 1)]
[(0, 0.82924613423585158), (1, 0.17075386576414842)]
manga review
[(1649, 1), (2945, 1)]
[(0, 0.82921175405940895), (1, 0.17078824594059105)]
line stars.â 
[(8784, 1), (4520, 1), (324, 1), (541, 1), (262, 1)]
[(0, 0.091006204955734332), (1, 0.9089937950442657)]
opening credit nature roll episode
[(8867, 1), (6780, 1)]
[(0, 0.17371426227338307), (1, 0.82628573772661695)]
stud

It turns out that going through a different number of passes does not change much. Although some of the coefficients are different, we see that the words themselves did not change much within the two topics. 

One thing that we do see, however, is that the topics seem to be very similar in scope. This is evidenced by the large number words occuring in both topics, such as "anime", "story", and "character". This also implies that the topics reviewers focuses on are largely the same, and it may be too difficult to distinguish between topics because they are so similar to each other.

However, we should also note that there was a warning saying that there is a sparse amount of data that we have. This is probably an indicator that there is not enough data to find distinct topics.

Regardless of that, we can still try to make some minor conclusions. For one, Topic 0 focuses more on the storyline or the plot of the anime in question whereas the other one could be focusing more on the series/episode as a whole. That is, one is more about specifics of an anime whereas another one could be an overall judgement of an anime. 

#Sentiment Analysis with Naive Bayes

First extract the adjectives in the sentences and create a RRD just like we did for the nouns. Then, create the adjective vocabulary ffor Naive Bayes Classifier. 

In [54]:
nbdatardd=sc.parallelize([ele[1] for ele in parseout])
nbdatardd.cache()
nbdatardd.take(10)

[[[u'short', u'different']],
 [[u'great']],
 [],
 [],
 [[u'anime', u'cliche-ridden', u'magical'],
  [u'magical', u'prepubescent', u'colorful', u'long', u'chinese'],
  [u'dear']],
 [[u'good'],
  [u'small'],
  [u'funny\xe2\xa0and', u'is\xe2\xa0an', u'alternate'],
  [u'hotspring']],
 [[u'entire', u'normal']],
 [[u'old', u'anime', u'heroic', u'evil'], [u'old'], [u'simple', u'short']],
 [[u'beautiful'],
  [u'anime', u'different'],
  [u'mysterious', u'dense'],
  [u'clear', u'consistent'],
  [u'hard-boiled', u'common', u'alternate', u'mystical']],
 [[u'original'], [u'particular', u'favorite', u'aforementioned']]]

In [56]:
adjvocabtups = (nbdatardd.flatMap(lambda l: l).flatMap(lambda word: word)
             .map(lambda word: (word, 1))
             .reduceByKey(lambda a, b: a + b)
             .map(lambda (x,y): x)
             .zipWithIndex()
).cache()
ldadatardd.flatMap(lambda word: word).take(5)
adjvocab  = adjvocabtups.collectAsMap()
len(adjvocab)

4840

First flatten the list of sentences as a list of all the adjectives in a certain review.

In [58]:
import itertools
Xarraypre=nbdatardd.map(lambda l: " ".join(list(itertools.chain.from_iterable(l))))
Xarray=Xarraypre.collect()
print Xarray[0]

short different


In [59]:
from sklearn.cross_validation import train_test_split
itrain, itest = train_test_split(xrange(len(Xarray)), train_size=0.7)
mask=np.ones(len(Xarray), dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)